<a href="https://colab.research.google.com/github/Peksyaji/Kaggle/blob/main/Klasifikasi%20Berita%20Hoax_GEMASTIK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Load Data

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/a3x-crypt/indo-covid19-news/main/data-v2.csv')
df.head()

,Unnamed: 0,text,hoax
0,0,Senin 27 April 2020 Sumatera Barat Lockdown,1
1,1,Anjuran Berbelanja Ketika Menjalankan “Social ...,0
2,2,Kesembuhan Harian Lampaui Penambahan Kasus Baru,0
3,3,[Top 5] Topik Teratas Periksa Fakta MAFINDO Pe...,1
4,4,Beradaptasi Kebiasaan Baru yang Aman COVID-19 ...,0


In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2496 entries, 0 to 2495
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2496 non-null   object
 1   hoax    2496 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 39.1+ KB


In [5]:
df.duplicated().sum()

0

In [6]:
df['hoax'].value_counts()

0    1811
1     685
Name: hoax, dtype: int64

Data yang memiliki label hoax memiliki proporsi sekitar 27,44% dari keseluruhan observasi. Jika meninjau dari https://developers.google.com/machine-learning/data-prep/construct/sampling-splitting/imbalanced-data, maka presentase tersebut termasuk dalam kategori data tidak seimbang yang ringan (mild imbalance data).

# Preprocessing Data

In [7]:
import re

In [8]:
stopword = pd.read_csv('https://raw.githubusercontent.com/listakurniawati/COVID-19-With-SVM/main/stopwords_id.csv?token=ARCQD7EZ55J4TUTAWYYLYOTAX3FTW')
stopword = np.append(stopword, 'top')
print('Jumlah kata dalam stopword: ', len(stopword))

def clean_text(teks):

    # Convert to lower case
    teks = teks.lower()
    # Clean www.* or https?://*
    teks = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',teks)
    #Remove punctuation
    teks = re.sub(r'[^\w\s]',' ', teks)
    #Replace #word with word
    teks = re.sub(r'#([^\s]+)', r'\1', teks)
    #Remove punctuation
    teks = re.sub(r'[^\w\s]',' ', teks)
    #Clean number
    teks = re.sub(r'[\d-]', '', teks)
    #Remove additional white spaces
    teks = re.sub('[\s]+', ' ', teks)
    #trim
    teks = teks.strip('\'"')
    # Clean per Words
    words = teks.split()
    tokens=[]
    for ww in words:
        #split repeated word
        for w in re.split(r'[-/\s]\s*', ww):
            #replace two or more with two occurrences
            pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
            w = pattern.sub(r"\1\1", w)
            #strip punctuation
            w = w.strip('\'"?,.')
            #check if the word cosists of two or more alphabets
            val = re.search(r"^[a-zA-Z][a-zA-Z][a-zA-Z]*$", w)
            #add tokens
            if(w in stopword or val is None):
                continue
            else:
                tokens.append(w.lower())

    teks = " ".join(tokens)
    return teks

print('Done!')

Jumlah kata dalam stopword:  759
Done!


In [9]:
df['teks_bersih'] = df['text'].apply(clean_text)
df.head()

,text,hoax,teks_bersih
0,Senin 27 April 2020 Sumatera Barat Lockdown,1,senin april sumatera barat lockdown
1,Anjuran Berbelanja Ketika Menjalankan “Social ...,0,anjuran berbelanja menjalankan social distancing
2,Kesembuhan Harian Lampaui Penambahan Kasus Baru,0,kesembuhan harian lampaui penambahan
3,[Top 5] Topik Teratas Periksa Fakta MAFINDO Pe...,1,topik teratas periksa fakta mafindo periode ma...
4,Beradaptasi Kebiasaan Baru yang Aman COVID-19 ...,0,beradaptasi kebiasaan aman covid produktif


# Feature Extraction

In [10]:
x = df['teks_bersih']
y = df['hoax']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(ngram_range=(1,1))
tf_idf.fit(x)

TfidfVectorizer()

In [12]:
# Melihat Jumlah Fitur
print(len(tf_idf.get_feature_names_out()))

3833


In [13]:
# Melihat fitur-fitur apa saja yang ada di dalam corpus
print(tf_idf.get_feature_names_out())

['aa' 'aadc' 'aamiin' ... 'zithromax' 'zona' 'zonasi']


In [14]:
X_tf_idf = tf_idf.transform(x).toarray()
X_tf_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Feature Selection

In [15]:
data_tf_idf = pd.DataFrame(X_tf_idf, columns=tf_idf.get_feature_names_out())
X = np.array(data_tf_idf)
y = np.array(y)

In [16]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

chi2_features = SelectKBest(chi2, k=3833)
X_kbest_features = chi2_features.fit_transform(X, y)

# Reduksi fitur
print('Banyak fitur semula:', X.shape[1])
print('Banyak fitur setelah reduksi:', X_kbest_features.shape[1])

Banyak fitur semula: 3833
Banyak fitur setelah reduksi: 3833


# Modelling

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_kbest_features, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

In [20]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

In [21]:
models = {'Linier SVC': svm.LinearSVC(C=5.0, random_state=42),
          'SCV': svm.SVC(C=5.0, random_state=42),
          'Random forest': RandomForestClassifier(random_state=42)
          }

In [22]:
from time import time

In [23]:
# Validasi silang model
for name, model in models.items():
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    predict_time = time()-start
    CV = KFold(n_splits = 5, shuffle = True, random_state = 42)
    scores = cross_val_score(model, X_train, y_train, cv = CV)
    print(model)
    print("\tWaktu latih: %0.3fs" % train_time)
    print("\tWaktu prediksi: %0.3fs" % predict_time)
    print('\t5-Fold Validation Scores:', scores)
    print('\tCross Validation Training Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

LinearSVC(C=5.0, random_state=42)
	Waktu latih: 0.044s
	Waktu prediksi: 0.005s
	5-Fold Validation Scores: [0.89       0.94486216 0.89223058 0.92982456 0.90726817]
	Cross Validation Training Accuracy: 0.913 (0.021)
SVC(C=5.0, random_state=42)
	Waktu latih: 8.492s
	Waktu prediksi: 2.340s
	5-Fold Validation Scores: [0.88       0.93233083 0.86716792 0.90726817 0.88721805]
	Cross Validation Training Accuracy: 0.895 (0.023)
RandomForestClassifier(random_state=42)
	Waktu latih: 2.787s
	Waktu prediksi: 0.031s
	5-Fold Validation Scores: [0.845      0.88972431 0.83458647 0.87969925 0.85714286]
	Cross Validation Training Accuracy: 0.861 (0.021)


In [24]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

In [25]:
# Metrik evaluasi
results = dict()
for name, model in models.items():
    model.fit(X_train, y_train)
    start = time()
    predicted = model.predict(X_test)
    predict_time = time()-start
    print('\nAlgorithm: ', name)
    print("\tWaktu prediksi: %0.3fs" % predict_time)
    res_cm = confusion_matrix(y_test, predicted)
    print('\tConfusion Matrix:\n', res_cm)
    res_acc = accuracy_score(y_test, predicted)
    print('\tAccuracy: ', res_acc)
    res_pres = precision_score(y_test, predicted, average='micro')
    print('\tPrecision: ', res_pres)
    res_recall = recall_score(y_test, predicted, average='micro')
    print('\tRecall Score: ', res_recall)
    res_f1 = f1_score(y_test, predicted, average='micro')
    print('\tF1 Score: ', res_f1)

    results[name]=res_acc


Algorithm:  Linier SVC
	Waktu prediksi: 0.003s
	Confusion Matrix:
 [[360  14]
 [ 25 101]]
	Accuracy:  0.922
	Precision:  0.922
	Recall Score:  0.922
	F1 Score:  0.922

Algorithm:  SCV
	Waktu prediksi: 1.980s
	Confusion Matrix:
 [[364  10]
 [ 34  92]]
	Accuracy:  0.912
	Precision:  0.912
	Recall Score:  0.912
	F1 Score:  0.912

Algorithm:  Random forest
	Waktu prediksi: 0.045s
	Confusion Matrix:
 [[364  10]
 [ 50  76]]
	Accuracy:  0.88
	Precision:  0.88
	Recall Score:  0.88
	F1 Score:  0.88


In [26]:
# hyperparameters tuning
parameter = {'loss':['hinge', 'squared_hinge'],
             'dual':[True, False],
             'multi_class':['ovr', 'crammer_singer'],
             'fit_intercept':[True, False],
             'class_weight':['balanced', None]
             }

grid = GridSearchCV(svm.LinearSVC(C=5.0, random_state=42), parameter)
model = grid.fit(X_train, y_train)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

{'class_weight': 'balanced', 'dual': True, 'fit_intercept': False, 'loss': 'squared_hinge', 'multi_class': 'ovr'} 

LinearSVC(C=5.0, class_weight='balanced', fit_intercept=False, random_state=42) 



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py", line 1223, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty

In [27]:
svc = svm.LinearSVC(C=5.0, class_weight='balanced', fit_intercept=False, random_state=42)
svc.fit(X_train, y_train)
pred = svc.predict(X_test)

cm = confusion_matrix(y_test, pred)
print('Confusion Matrix:\n', cm)
print(classification_report(y_test, pred))

Confusion Matrix:
 [[357  17]
 [ 23 103]]
              precision    recall  f1-score   support

           0       0.94      0.95      0.95       374
           1       0.86      0.82      0.84       126

    accuracy                           0.92       500
   macro avg       0.90      0.89      0.89       500
weighted avg       0.92      0.92      0.92       500

